In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .master("local[*]")
    .getOrCreate()    
)

read images using image format and glob pattern in path

In [2]:
import os

df = spark.read.format("image").load(os.path.join("/opt/workspace", "media/images/catsAndDogs40/train/*/*.jpg"))
df.show(5)

+--------------------+
|               image|
+--------------------+
|{file:///opt/work...|
|{file:///opt/work...|
|{file:///opt/work...|
|{file:///opt/work...|
|{file:///opt/work...|
+--------------------+


it appears to show a single column as struct object, let's look at the schema

In [3]:
df.printSchema()

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)


okay, so let's expand image dictionary into columns

In [4]:
df.select("image.*").show(5)

+--------------------+------+-----+---------+----+--------------------+
|              origin|height|width|nChannels|mode|                data|
+--------------------+------+-----+---------+----+--------------------+
|file:///opt/works...|   160|  160|        3|  16|[0B 17 17 13 20 1...|
|file:///opt/works...|   160|  160|        3|  16|[32 3E 3E 33 42 3...|
|file:///opt/works...|   160|  160|        3|  16|[F2 EF EB F3 F0 E...|
|file:///opt/works...|   160|  160|        3|  16|[FA C5 82 FA C5 8...|
|file:///opt/works...|   160|  160|        3|  16|[40 53 58 54 6A 6...|
+--------------------+------+-----+---------+----+--------------------+


extract image features

In [6]:
import pyspark.sql.functions as F
from pyspark.sql.pandas.functions import pandas_udf
import pyspark.sql.types as T
import pandas as pd
from PIL import ImageStat, Image

@pandas_udf(returnType=T.ArrayType(T.FloatType()))
def mean_pixel_levels(height: pd.Series, width: pd.Series, bytes: pd.Series) -> pd.Series:
    mode = 'YCbCr'
    return pd.Series(ImageStat.Stat(Image.frombytes(mode, (h,w), b)).mean for h,w,b in list(zip(height, width, bytes)))
    

features_df = df.select("image.*")
features_df = features_df.select(
    F.reverse(F.split(F.col("origin"),"/")).getItem(0).alias("file_name"),
    F.col("height"),F.col("width"),
    mean_pixel_levels(F.col("height"),F.col("width"),F.col("data")).alias("mean_pixel_levels")
)
features_df.show(5)


+---------+------+-----+--------------------+
|file_name|height|width|   mean_pixel_levels|
+---------+------+-----+--------------------+
|   17.jpg|   160|  160|[105.68774, 122.5...|
|   28.jpg|   160|  160|[90.60562, 100.55...|
|   20.jpg|   160|  160|[107.34871, 119.9...|
|    1.jpg|   160|  160|[117.33078, 136.5...|
|   24.jpg|   160|  160|[65.62848, 96.271...|
+---------+------+-----+--------------------+
